# Exploring `build_bus_regions` outputs

### Jupyter notebook needs to be improved

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import pypsa
import shutil
from rasterio.plot import show

import os
import sys

# change current directory to pypsa-earth folder
os.chdir("../../pypsa-earth")

# Input

In [ ]:
# Load network
network_path = os.getcwd() + "/networks/base.nc"
n = pypsa.Network(network_path)
n

In [ ]:
buses = n.buses
# Adding geometries from x and y coordinates
buses["geometry"] = gpd.points_from_xy(buses.lon, buses.lat)
buses = gpd.GeoDataFrame(buses, crs="epsg:4326")

continent = gpd.read_file("resources/shapes/country_shapes.geojson")
off_shore = gpd.read_file("resources/shapes/offshore_shapes.geojson")

ax = continent.plot(figsize=(10, 10))
off_shore.plot(ax=ax)
buses.plot(ax=ax, color="orange", markersize=1)

In [ ]:
buses[buses.country == "NG"]

# Output

In [ ]:
continent = gpd.read_file("resources/bus_regions/regions_onshore.geojson")
off_shore = gpd.read_file("resources/bus_regions/regions_offshore.geojson")

ax = continent.plot(figsize=(10, 10))
off_shore.plot(ax=ax, color="red")

In [ ]:
c_b = n.buses.country == "NG"
n.buses.substation_off
b = n.buses.substation_off & c_b
b.sum()

In [ ]:
c_b.sum()

In [ ]:
n.buses.groupby("country").sum()

In [ ]:
from shapely.geometry import LineString, Point

lines = n.lines[n.lines.country == "NG"].copy()
# lines = n.lines.copy()
lines["geometry"] = [
    LineString([Point(x.bus0_lon, x.bus0_lat), Point(x.bus1_lon, x.bus1_lat)])
    for index, x in lines.iterrows()
    if (x.bus0_lat is not None)
    and (x.bus0_lon is not None)
    and (x.bus1_lat is not None)
    and (x.bus1_lon is not None)
]
lines = gpd.GeoDataFrame(geometry=lines.geometry, crs="epsg:4326")

continent = gpd.read_file("resources/shapes/country_shapes.geojson")
off_shore = gpd.read_file("resources/shapes/offshore_shapes.geojson")

ax = continent.plot(figsize=(10, 10))
off_shore.plot(ax=ax)
c_code = "CD"
# lines[lines.country==c_code].plot(ax=ax, color="yellow")
# buses[buses.country==c_code].plot(ax=ax, color = "orange", markersize = 1)
lines.plot(ax=ax, color="yellow")
buses.plot(ax=ax, color="orange", markersize=1)